In [ ]:
import pandas as pd
import numpy as np 
from sklearn.cluster import KMeans
from itertools import combinations    #分群維度的排列組合
import plotly.graph_objects as go

In [ ]:
file_name='month1_python'  #file name
path = r'C:\Users\user\分群結果/'  #電腦路徑
min_cluster = 2  #最小群數
max_cluster = 11 #最大群數

m01 = pd.read_csv(file_name+'.csv',encoding='big5')
m01

In [ ]:
m001 = m01.rename(columns={'與今年1月日平均差異':'A','與去年1月日平均差異':'B','與今年日平均差異':'C'}) #寫入檔名會比較方便
m001

In [ ]:
data = list(m001.columns)   #分群排列組合
combin = list([c for i in range(1, len(data)+1) for c in combinations(data, i)])
combin

In [ ]:
permutation = []                #資料轉型
for i in range(len(combin)):
    permutation.append(list(combin[i]))
permutation

In [ ]:
for i in range(len(permutation)):
    data = m001[permutation[i]]
    Silhouette = []
    for j in range(min_cluster,max_cluster):
        kmeans = KMeans(init='random',n_clusters=j,random_state=0)
        kmeans.fit(data)
        Silhouette.append(kmeans.inertia_)
        if j == 10:
            Silhouette_vertify = []
            for k in range(len(Silhouette)):
                if len(Silhouette) - k != 1:
                    Silhouette_vertify.append(Silhouette[k+1] - Silhouette[k])
                else:
                    Silhouette_vertify.append(Silhouette[k] - Silhouette[k-1])
            Silhouette_data = {'輪廓系數':Silhouette,'最佳分群判斷':Silhouette_vertify}
            Silhouette_vertify_data = pd.DataFrame(data=Silhouette_data)
            Silhouette_vertify_data.to_csv(path+str(permutation[i])+'_輪廓系數'+'.csv', index = False, header=True,encoding='utf-8-sig')
    fig = go.Figure(data=go.Scatter(x=list(range(min_cluster,max_cluster)),y=Silhouette))
    fig.update_layout(xaxis_title='Number of clusters',yaxis_title='Distortion',font={'size':22,'family':'sans-serif'})
    fig.write_image(path+str(permutation[i])+'.png')
    for l in range(min_cluster,max_cluster):
        kmeans = KMeans(init='random',n_clusters=l,random_state=0)
        kmeans.fit(data)
        labels = pd.Series(kmeans.labels_,name='cluster_number')
        data['分群數量'+str(l)] = labels
    data.to_csv(path+str(permutation[i])+'.csv', index = False, header=True,encoding='utf-8-sig')